# 🤗 End-to-end distilabel example with Inference Endpoints and Notus

[distilabel](https://github.com/argilla-io/distilabel) is an AI Feedback (AIF) framework that can generate and label datasets using LLMs, and can be used for many different use cases. Implemented with robustness, efficiency and scalability in mind, it allows anyone to build their own synthetic datasets that can be used in many different scenarios. This tutorials shows and end-to-end example in which we will create a model expert in the new AI Act, to which we can make different types of questions and requests. 

The LLM model that we will fine-tune for this is [Notus 7B](https://argilla.io/blog/notus7b/), a fine-tuned version of Zephyr 7B that uses Direct Preference Optimization (DPO) and AIF techniques to outperfom its foundation model in several benchmarks, and is completely open-source. 

## Introduction

Let's start by installing the required depencies to run distilabel, Argilla and the rest of the packages used in the tutorial

In [1]:
#%pip install argilla distilabel farm-haystack pip install "distilabel[hf-inference-endpoints]"

### Running Argilla

For this tutorial, you can use Argilla to visualize and annotate the different datasets created by distilabel. There are two main options for deploying and running Argilla:

**Deploy Argilla on Hugging Face Spaces:** If you want to run tutorials with external notebooks (e.g., Google Colab) and you have an account on Hugging Face, you can deploy Argilla on Spaces with a few clicks:

[![deploy on spaces](https://huggingface.co/datasets/huggingface/badges/raw/main/deploy-to-spaces-lg.svg)](https://huggingface.co/new-space?template=argilla/argilla-template-space)

For details about configuring your deployment, check the [official Hugging Face Hub guide](https://huggingface.co/docs/hub/spaces-sdks-docker-argilla).

**Launch Argilla using Argilla's quickstart Docker image**: This is the recommended option if you want [Argilla running on your local machine](../../getting_started/quickstart.ipynb). Note that this option will only let you run the tutorial locally and not with an external notebook service.

For more information on deployment options, please check the Deployment section of the documentation.

<div class="alert alert-info">

Tip

This tutorial is a Jupyter Notebook. There are two options to run it:

- Use the Open in Colab button at the top of this page. This option allows you to run the notebook directly on Google Colab. Don't forget to change the runtime type to GPU for faster model training and inference.
- Download the .ipynb file by clicking on the View source link at the top of the page. This option allows you to download the notebook and run it on your local machine or on a Jupyter notebook tool of your choice.
</div>

### Import dependencies

The main dependencies for this tutorial are distilabel for creating the synthetic datasets and Argilla for visualizing and annotating these datasets, and also for fine-tuning our model. The package [Haystack](https://haystack.deepset.ai/) is used to creates batches from the original PDF document we want to create our datasets from. 

In [2]:
import os
import time
from typing import Dict

import argilla as rg

from distilabel.llm import InferenceEndpointsLLM
from distilabel.pipeline import Pipeline, pipeline
from distilabel.tasks import Llama2TextGenerationTask, SelfInstructTask, Prompt

from datasets import Dataset
from haystack.nodes import PDFToTextConverter, PreProcessor

If you are running Argilla using the Docker quickstart image or Hugging Face Spaces, you need to init the Argilla client with the URL and API_KEY:

In [3]:
# Replace api_url with the url to your HF Spaces URL if using Spaces
# Replace api_key if you configured a custom API key
rg.init(
    api_url="https://ignacioct-argilla.hf.space",
    api_key="owner.apikey",
    workspace="admin"
)

/Users/ignacio/Documents/recognai/argilla/src/argilla/client/client.py:182: UserWarning: You're connecting to Argilla Server 1.19.0 using a different client version (1.20.0).
This may lead to potential compatibility issues during your experience.
To ensure a seamless and optimized connection, we highly recommend aligning your client version with the server version.
  warnings.warn(


Additionally, we need to provide our HuggingFace and OpenAI accest token. To later instatiate an `InferenceEndpointsLLM` object, we need to pass as parameters the HF Inference Endpoint name and the HF namespace. One very convenient way to do so is also through environment variables.

In [4]:
os.environ["HF_TOKEN"] = "hf_"
os.environ["HF_INFERENCE_ENDPOINT_NAME"] = "aws-notus-7b-v1-3184"
os.environ["HF_NAMESPACE"] = "argilla"
os.environ["OPENAI_API_KEY"] = "sk-"

## Setting up an inference endpoint with Notus

To kickstart this tutorial, let's see how to set up and endpoint for our Notus model. A HuggingFace endpoint is a service provided by HuggingFace that allows you to deploy and host your machine learning models for inference. This way, we'll have faster inference times, as these models will not run in our personal machines, but in HuggingFace servers. The endpoint of choice has a [Notus 7B instance](https://ui.endpoints.huggingface.co/argilla/endpoints/aws-notus-7b-v1-4052) running.

Let's see a quick example of how to use an inference endpoint. We have prepared an easy `Llama2QuestionAnsweringTask` to ask question to the model, in a very similar way as we talk with the LLMs using chatbots.

In [5]:
class Llama2QuestionAnsweringTask(Llama2TextGenerationTask):
    def generate_prompt(self, question: str) -> str:
        return Prompt(
            system_prompt=self.system_prompt,
            formatted_prompt=question,
        ).format_as("llama2")  # type: ignore

    def parse_output(self, output: str) -> Dict[str, str]:
        return {"answer": output.strip()}

    def input_args_names(self) -> list[str]:
        return ["question"]

    def output_args_names(self) -> list[str]:
        return ["answer"]

The `llm` is an object of the `InferenceEndpointsLLM` class, and through it we can start generating answers to question using the `llm.generate()` method.

In [6]:
llm = InferenceEndpointsLLM(
    endpoint_name=os.getenv("HF_INFERENCE_ENDPOINT_NAME"),  # type: ignore
    endpoint_namespace=os.getenv("HF_NAMESPACE"),  # type: ignore
    token=os.getenv("HF_TOKEN") or None,
    task=Llama2QuestionAnsweringTask(),
)

In [7]:
generation = llm.generate([{"question": "What's the second most populated city in Denmark?"}])
generation[0][0]["parsed_output"]["answer"]

'The second most populated city in Denmark is Aarhus, with a population of around 340,000 people. It is located on the east coast of Jutland, and is known for its vibrant cultural scene, beautiful beaches, and historic landmarks. Aarhus is also home to Aarhus University, one of the largest universities in Scandinavia.'

The endpoint is working! We now can do inference through the Inference Endpoint.

## Downloading the AI Act PDF document

As we want an expert model of the new AI Act promoted by the European Union, we firstly need to download the PDF document itself.

In [8]:
!wget https://artificialintelligenceact.eu/wp-content/uploads/2021/08/The-AI-Act.pdf

--2023-12-19 12:53:07--  https://artificialintelligenceact.eu/wp-content/uploads/2021/08/The-AI-Act.pdf
Resolving artificialintelligenceact.eu (artificialintelligenceact.eu)... 173.255.227.216
Connecting to artificialintelligenceact.eu (artificialintelligenceact.eu)|173.255.227.216|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1351521 (1,3M) [application/pdf]
Saving to: ‘The-AI-Act.pdf’

The-AI-Act.pdf      100%[===================>]   1,29M   950KB/s    in 1,4s    

2023-12-19 12:53:09 (950 KB/s) - ‘The-AI-Act.pdf’ saved [1351521/1351521]



Once we have it in our working directory, we can use Haystack's converter and pipeline features to extract the textual data, clean it and divide it in different batches. Afterwards, these batches will be used to start creating synthetic instructions.

In [9]:
converter = PDFToTextConverter(
    remove_numeric_tables=True,
    valid_languages=["en"]
)

doc = converter.convert(file_path="The-AI-Act.pdf", meta=None)[0]

pdftotext version 4.04 [www.xpdfreader.com]
Copyright 1996-2022 Glyph & Cog, LLC


In [10]:
preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="word",
    split_length=150,
    split_respect_sentence_boundary=True,
)
docs = preprocessor.process([doc])
print(f"Number of input documents: 1\nNumber of output documents: {len(docs)}")


Preprocessing:   0%|          | 0/1 [00:00<?, ?docs/s]

[12/19/23 12:53:12] WARNING  WARNING:haystack.nodes.preprocessor.preprocessor:We found one or   ]8;id=138802;file:///Users/ignacio/opt/anaconda3/envs/distilabel/lib/python3.11/site-packages/haystack/nodes/preprocessor/preprocessor.py\preprocessor.py]8;;\:]8;id=586780;file:///Users/ignacio/opt/anaconda3/envs/distilabel/lib/python3.11/site-packages/haystack/nodes/preprocessor/preprocessor.py#516\516]8;;\
                             more sentences whose split count is higher than the split length.                     

Preprocessing: 100%|██████████| 1/1 [00:00<00:00,  3.69docs/s]

Number of input documents: 1
Number of output documents: 355


Let's take a look at the batches:

In [11]:
inputs = [doc.content for doc in docs]
inputs[0:5]

['EN EN\nEUROPEAN\nCOMMISSION\nProposal for a\nREGULATION OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL\nLAYING DOWN HARMONISED RULES ON ARTIFICIAL INTELLIGENCE\n(ARTIFICIAL INTELLIGENCE ACT) AND AMENDING CERTAIN UNION\nLEGISLATIVE ACTS\x0cEN\nEXPLANATORY MEMORANDUM\n1. CONTEXT OF THE PROPOSAL\n1.1. Reasons for and objectives of the proposal\nThis explanatory memorandum accompanies the proposal for a Regulation laying down\nharmonised rules on artificial intelligence (Artificial Intelligence Act). Artificial Intelligence\n(AI) is a fast evolving family of technologies that can bring a wide array of economic and\nsocietal benefits across the entire spectrum of industries and social activities. By improving\nprediction, optimising operations and resource allocation, and personalising service delivery,\nthe use of artificial intelligence can support socially and environmentally beneficial outcomes\nand provide key competitive advantages to companies and the European economy. ',
 'Such\nac

## Generating instructions with SelfInstructTask

With out Inference Endpoint up and running, we should be able to generate instructions with distilabel. These instructions, made by the LLM through our endpoint, will form an instruction dataset. For this example, we are using a subset of all the batches generated in the section above, to be gentle on performance. 

In [12]:
instructions_dataset = Dataset.from_dict({
    "input": inputs[0:50]
})

instructions_dataset

Dataset({
    features: ['input'],
    num_rows: 50
})

With the `SelfInstructTask` class, we can generate a Self-Instruct specitification for building the prompts, as done in the [Self-Instruct paper](https://arxiv.org/abs/2212.10560). This Self-Instruct framework bootstraps LLM generations. We will pass as a parameter an application description to specify the behaviour of the model. 

In [13]:
instructions_task = SelfInstructTask(
    application_description="A assistant that can answer questions about the AI Act made by the European Union."
)

Let's now define a generator, passing the `SelfInstructTask` object, and create a `Pipeline` object.

In [14]:
instructions_generator = InferenceEndpointsLLM(
    endpoint_name=os.getenv("HF_INFERENCE_ENDPOINT_NAME"),  # type: ignore
    endpoint_namespace=os.getenv("HF_NAMESPACE"),  # type: ignore
    token=os.getenv("HF_TOKEN") or None,
    task=instructions_task,
)

instructions_pipeline = Pipeline(
    generator=instructions_generator
)

Our pipeline is ready to be used to generate instructions. Let's do it!

In [15]:
generated_instructions = instructions_pipeline.generate(dataset=instructions_dataset, num_generations=1, batch_size=8)

[12/19/23 12:53:24] INFO     INFO:distilabel:Executing dry-run...                                   ]8;id=453346;file:///Users/ignacio/.local/lib/python3.11/site-packages/distilabel/pipeline.py\pipeline.py]8;;\:]8;id=621782;file:///Users/ignacio/.local/lib/python3.11/site-packages/distilabel/pipeline.py#751\751]8;;\

                    INFO     INFO:distilabel:Processing batch 1 of 1...                             ]8;id=93172;file:///Users/ignacio/.local/lib/python3.11/site-packages/distilabel/pipeline.py\pipeline.py]8;;\:]8;id=454265;file:///Users/ignacio/.local/lib/python3.11/site-packages/distilabel/pipeline.py#644\644]8;;\

                    INFO     INFO:distilabel:Calling generator for batch 1...                       ]8;id=989407;file:///Users/ignacio/.local/lib/python3.11/site-packages/distilabel/pipeline.py\pipeline.py]8;;\:]8;id=587669;file:///Users/ignacio/.local/lib/python3.11/site-packages/distilabel/pipeline.py#648\648]8;;\

/Users/ignacio/.local/lib/python3.11/site-packages/distilabel/llm/huggingface/inference_endpoints.py:177: UserWarning: No `prompt_format` has been specified and no `default_format` is set, so the prompt will be concatenated with a line-break and no specific formatting by default.
  prompts = self._generate_prompts(


Flattening the indices:   0%|          | 0/1 [00:00<?, ? examples/s]

...

                    INFO     INFO:distilabel:Calling generator for batch 6...                       ]8;id=832610;file:///Users/ignacio/.local/lib/python3.11/site-packages/distilabel/pipeline.py\pipeline.py]8;;\:]8;id=18020;file:///Users/ignacio/.local/lib/python3.11/site-packages/distilabel/pipeline.py#648\648]8;;\

[12/19/23 12:57:08] INFO     INFO:distilabel:Processing batch 7 of 7...                             ]8;id=574676;file:///Users/ignacio/.local/lib/python3.11/site-packages/distilabel/pipeline.py\pipeline.py]8;;\:]8;id=577407;file:///Users/ignacio/.local/lib/python3.11/site-packages/distilabel/pipeline.py#644\644]8;;\

                    INFO     INFO:distilabel:Calling generator for batch 7...                       ]8;id=155604;file:///Users/ignacio/.local/lib/python3.11/site-packages/distilabel/pipeline.py\pipeline.py]8;;\:]8;id=594852;file:///Users/ignacio/.local/lib/python3.11/site-packages/distilabel/pipeline.py#648\648]8;;\

Flattening the indices:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Our pipeline has succesfully generated instructions given the topics and the behaviour passed as input. Let's gather all those instructions and see how the look.

In [16]:
instructions = []
for generations in generated_instructions["generations"]:
    for generation in generations:
        instructions.extend(generation)

print(f"Number of generated instructions: {len(instructions)}")

for instruction in instructions[:5]:
    print(instruction)

Number of generated instructions: 242
1. What are the reasons for and objectives of the proposal for a Regulation laying down harmonised rules on artificial intelligence?
2. How can artificial intelligence improve prediction, optimise operations and resource allocation, and personalise service delivery?
3. What benefits can artificial intelligence bring to the European economy and society as a whole?
4. How can the use of artificial intelligence support socially and environmentally beneficial outcomes?
5. What competitive advantages can companies gain from using artificial intelligence?


These initial intructions form our instruction dataset. Following the human-in-the-loop approach, we should push the instructions to Argilla to visualize them and be able to rank them in terms of quality. Those annotations would make quality data, ensuring a better performance of the final model. Nevertheless, this step is optional.

### Pushing the instruction dataset to Argilla to visualize and annotate.

In [17]:
instructions_rg_dataset = generated_instructions.to_argilla()
instructions_rg_dataset.push_to_argilla(name=f"notus_AI_instructions")

Output()

[12/19/23 12:59:45] INFO     INFO:argilla.client.feedback.dataset.local.mixins:✓ Dataset succesfully  ]8;id=708190;file:///Users/ignacio/Documents/recognai/argilla/src/argilla/client/feedback/dataset/local/mixins.py\mixins.py]8;;\:]8;id=980232;file:///Users/ignacio/Documents/recognai/argilla/src/argilla/client/feedback/dataset/local/mixins.py#281\281]8;;\
                             pushed to Argilla                                                                     

                    INFO     INFO:argilla.client.feedback.dataset.local.mixins:RemoteFeedbackDataset( ]8;id=693154;file:///Users/ignacio/Documents/recognai/argilla/src/argilla/client/feedback/dataset/local/mixins.py\mixins.py]8;;\:]8;id=931572;file:///Users/ignacio/Documents/recognai/argilla/src/argilla/client/feedback/dataset/local/mixins.py#282\282]8;;\
                                id=92c49ce5-4792-40f6-a8f5-1f1bba8f851c                                            
                                name=notus_AI_instructions                                                         
                                workspace=Workspace(id=3ccf7c31-bab2-4091-8d01-c0ffa6e605db,                       
                             name=admin, inserted_at=2023-12-18 08:21:20.790817,                                   
                             updated_at=2023-12-18 08:21:20.790817)                                                
                                url=https://ignacioct-argilla.hf.space/dataset/92c49ce5-4792-40f6-a8f              
                             5-1f1bba8f851c/annotation-mode                                                        
                                fields=[RemoteTextField(id=UUID('dbedc3d9-5dd9-441c-83ae-3b699e23f594              
                             '), client=None, name='input', title='input', required=True,                          
                             type='text', use_markdown=False),                                                     
                             RemoteTextField(id=UUID('f1c0d914-9510-4aa2-91ed-cc83d69bbed4'),                      
                             client=None, name='generations-1', title='generations-1', required=True,              
                             type='text', use_markdown=False)]                                                     
                                questions=[RemoteRatingQuestion(id=UUID('c4040ccc-6ef2-4649-ac29-1774              
                             80b98481'), client=None, name='generations-1-rating', title='How would                
                             you rate the generation at `generations-1`?', description=None,                       
                             required=True, type='rating', values=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])]                
                                guidelines=None                                                                    
                                metadata_properties=[RemoteIntegerMetadataProperty(id=UUID('e7dec763-              
                             a118-4867-b116-35f936e48736'), client=<httpx.Client object at                         
                             0x1a0b34790>, name='length-input', title='length-input',                              
                             visible_for_annotators=True, type='integer', min=None, max=None),                     
                             RemoteIntegerMetadataProperty(id=UUID('fc600506-25f8-4e46-8f11-c2a460e94              
                             3a5'), client=<httpx.Client object at 0x1a0b34790>,                                   
                             name='length-generations-1', title='length-generations-1',                            
                             visible_for_annotators=True, type='integer', min=None, max=None)]                     
                             )                                                                                     

RemoteFeedbackDataset(
   id=92c49ce5-4792-40f6-a8f5-1f1bba8f851c
   name=notus_AI_instructions
   workspace=Workspace(id=3ccf7c31-bab2-4091-8d01-c0ffa6e605db, name=admin, inserted_at=2023-12-18 08:21:20.790817, updated_at=2023-12-18 08:21:20.790817)
   url=https://ignacioct-argilla.hf.space/dataset/92c49ce5-4792-40f6-a8f5-1f1bba8f851c/annotation-mode
   fields=[RemoteTextField(id=UUID('dbedc3d9-5dd9-441c-83ae-3b699e23f594'), client=None, name='input', title='input', required=True, type='text', use_markdown=False), RemoteTextField(id=UUID('f1c0d914-9510-4aa2-91ed-cc83d69bbed4'), client=None, name='generations-1', title='generations-1', required=True, type='text', use_markdown=False)]
   questions=[RemoteRatingQuestion(id=UUID('c4040ccc-6ef2-4649-ac29-177480b98481'), client=None, name='generations-1-rating', title='How would you rate the generation at `generations-1`?', description=None, required=True, type='rating', values=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])]
   guidelines=None
   metadat

## Generate a Preference Dataset using an Ultrafeedback text quality task.

Once we have our instruction dataset, we are going to create a preference dataset through the UltraFeedback text quality task. This is a type of task used in NLP used to evaluate the quality of text generated; our goal is to provide detailed feedback on the quality of the generated text, beyond a binary label.

Our `pipeline()` method allows us to create a `Pipeline` instance with the provided LLMs for a given task, which is useful whenever you want to use a pre-defined or custom `Pipeline` for a given task. We will specify our task and subtask, the generator we want to use (in this case, one based in a Llama2 Text Generator Task) and our OpenAI API key.

In [18]:
preference_pipeline = pipeline(
    "preference",
    "text-quality",
    generator=InferenceEndpointsLLM(
        endpoint_name=os.getenv("HF_INFERENCE_ENDPOINT_NAME"),  # type: ignore
        endpoint_namespace=os.getenv("HF_NAMESPACE", None),
        task=Llama2TextGenerationTask(),
        max_new_tokens=256,
        num_threads=2,
        temperature=0.3,
    ),
    max_new_tokens=256,
    num_threads=2,
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0.0,
)

[12/19/23 12:59:51] INFO     INFO:distilabel:Since no `labeller` was provided, `OpenAILLM` will be  ]8;id=94685;file:///Users/ignacio/.local/lib/python3.11/site-packages/distilabel/pipeline.py\pipeline.py]8;;\:]8;id=261193;file:///Users/ignacio/.local/lib/python3.11/site-packages/distilabel/pipeline.py#857\857]8;;\
                             used as the default labeller with `UltraFeedback`.                                    

We also need to retrieve our instruction dataset from Argilla, as it will be the input of this pipeline.

In [22]:
remote_dataset = rg.FeedbackDataset.from_argilla("notus_AI_instructions", workspace="admin")
instructions_dataset = remote_dataset.pull(max_records=100) # get first 100 records

instructions_dataset = instructions_dataset.format_as("datasets")
instructions_dataset

Dataset({
    features: ['input', 'generations-1', 'generations-1-rating', 'generations-1-rating-suggestion', 'generations-1-rating-suggestion-metadata', 'external_id', 'metadata'],
    num_rows: 50
})

Now, let's build a dataset by using the pipeline we just created, and the topics from which our instructions were generated. 

In [23]:
preference_dataset = preference_pipeline.generate(
    instructions_dataset,  # type: ignore
    num_generations=2,
    batch_size=8,
    enable_checkpoints=True,
    display_progress_bar=True,
)

[12/19/23 13:05:31] INFO     INFO:distilabel:Executing dry-run...                                   ]8;id=276044;file:///Users/ignacio/.local/lib/python3.11/site-packages/distilabel/pipeline.py\pipeline.py]8;;\:]8;id=4580;file:///Users/ignacio/.local/lib/python3.11/site-packages/distilabel/pipeline.py#751\751]8;;\

/Users/ignacio/.local/lib/python3.11/site-packages/distilabel/pipeline.py:757: UserWarning: Provided `num_generations=1` which implies that the `generator` LLM will just run once, while the `labelling` LLM expects to receive a list of N inputs to label, where N is > 1. If this is not intended, make sure to set `num_generations` to a value higher or equal to 2.
  _ = self._generate(


                    INFO     INFO:distilabel:Processing batch 1 of 1...                             ]8;id=785250;file:///Users/ignacio/.local/lib/python3.11/site-packages/distilabel/pipeline.py\pipeline.py]8;;\:]8;id=26939;file:///Users/ignacio/.local/lib/python3.11/site-packages/distilabel/pipeline.py#644\644]8;;\

                    INFO     INFO:distilabel:Calling generator for batch 1...                       ]8;id=692581;file:///Users/ignacio/.local/lib/python3.11/site-packages/distilabel/pipeline.py\pipeline.py]8;;\:]8;id=500556;file:///Users/ignacio/.local/lib/python3.11/site-packages/distilabel/pipeline.py#648\648]8;;\

[12/19/23 13:05:33] INFO     INFO:distilabel:Calling labeller for batch 1...                        ]8;id=865707;file:///Users/ignacio/.local/lib/python3.11/site-packages/distilabel/pipeline.py\pipeline.py]8;;\:]8;id=790516;file:///Users/ignacio/.local/lib/python3.11/site-packages/distilabel/pipeline.py#674\674]8;;\

...

                    INFO     INFO:distilabel:Calling generator for batch 6...                       ]8;id=801364;file:///Users/ignacio/.local/lib/python3.11/site-packages/distilabel/pipeline.py\pipeline.py]8;;\:]8;id=852727;file:///Users/ignacio/.local/lib/python3.11/site-packages/distilabel/pipeline.py#648\648]8;;\

[12/19/23 13:08:34] INFO     INFO:distilabel:Calling labeller for batch 6...                        ]8;id=593887;file:///Users/ignacio/.local/lib/python3.11/site-packages/distilabel/pipeline.py\pipeline.py]8;;\:]8;id=489580;file:///Users/ignacio/.local/lib/python3.11/site-packages/distilabel/pipeline.py#674\674]8;;\

[12/19/23 13:08:45] INFO     INFO:distilabel:Calling labeller for batch 7...                        ]8;id=278915;file:///Users/ignacio/.local/lib/python3.11/site-packages/distilabel/pipeline.py\pipeline.py]8;;\:]8;id=287635;file:///Users/ignacio/.local/lib/python3.11/site-packages/distilabel/pipeline.py#674\674]8;;\

Flattening the indices:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Let's take a look at an instance of the preference dataset

In [24]:
preference_dataset[0]

{'input': 'EN EN\nEUROPEAN\nCOMMISSION\nProposal for a\nREGULATION OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL\nLAYING DOWN HARMONISED RULES ON ARTIFICIAL INTELLIGENCE\n(ARTIFICIAL INTELLIGENCE ACT) AND AMENDING CERTAIN UNION\nLEGISLATIVE ACTS\x0cEN\nEXPLANATORY MEMORANDUM\n1. CONTEXT OF THE PROPOSAL\n1.1. Reasons for and objectives of the proposal\nThis explanatory memorandum accompanies the proposal for a Regulation laying down\nharmonised rules on artificial intelligence (Artificial Intelligence Act). Artificial Intelligence\n(AI) is a fast evolving family of technologies that can bring a wide array of economic and\nsocietal benefits across the entire spectrum of industries and social activities. By improving\nprediction, optimising operations and resource allocation, and personalising service delivery,\nthe use of artificial intelligence can support socially and environmentally beneficial outcomes\nand provide key competitive advantages to companies and the European economy.',
 '

## Setting up an Argilla HF Space to upload the resulting dataset.

In [25]:
# Uploading the Preference Dataset
preference_rg_dataset = preference_dataset.to_argilla()
preference_rg_dataset.push_to_argilla(name=f"notus_AI_preference")

Output()

[12/19/23 13:10:35] INFO     INFO:argilla.client.feedback.dataset.local.mixins:✓ Dataset succesfully  ]8;id=952946;file:///Users/ignacio/Documents/recognai/argilla/src/argilla/client/feedback/dataset/local/mixins.py\mixins.py]8;;\:]8;id=791902;file:///Users/ignacio/Documents/recognai/argilla/src/argilla/client/feedback/dataset/local/mixins.py#281\281]8;;\
                             pushed to Argilla                                                                     

                    INFO     INFO:argilla.client.feedback.dataset.local.mixins:RemoteFeedbackDataset( ]8;id=708921;file:///Users/ignacio/Documents/recognai/argilla/src/argilla/client/feedback/dataset/local/mixins.py\mixins.py]8;;\:]8;id=230650;file:///Users/ignacio/Documents/recognai/argilla/src/argilla/client/feedback/dataset/local/mixins.py#282\282]8;;\
                                id=667c5d11-5620-4d89-b3d4-03ba90278766                                            
                                name=notus_AI_preference                                                           
                                workspace=Workspace(id=3ccf7c31-bab2-4091-8d01-c0ffa6e605db,                       
                             name=admin, inserted_at=2023-12-18 08:21:20.790817,                                   
                             updated_at=2023-12-18 08:21:20.790817)                                                
                                url=https://ignacioct-argilla.hf.space/dataset/667c5d11-5620-4d89-b3d              
                             4-03ba90278766/annotation-mode                                                        
                                fields=[RemoteTextField(id=UUID('a60fd7af-02b3-4e7b-8522-41c2c177b711              
                             '), client=None, name='input', title='input', required=True,                          
                             type='text', use_markdown=False),                                                     
                             RemoteTextField(id=UUID('85f037c3-75fe-4aa8-91a1-4a4863e4dcb7'),                      
                             client=None, name='generations-1', title='generations-1', required=True,              
                             type='text', use_markdown=False),                                                     
                             RemoteTextField(id=UUID('68262ba8-5141-4e99-bab3-a48d4234bab2'),                      
                             client=None, name='generations-2', title='generations-2', required=True,              
                             type='text', use_markdown=False)]                                                     
                                questions=[RemoteRatingQuestion(id=UUID('d630657a-575c-4716-a1bb-2999              
                             35bcf076'), client=None, name='generations-1-rating', title="What's the               
                             rating for generations-1?", description=None, required=True,                          
                             type='rating', values=[1, 2, 3, 4, 5]),                                               
                             RemoteRatingQuestion(id=UUID('a63117b2-46f3-4b3c-bc3a-9da2816a12b3'),                 
                             client=None, name='generations-2-rating', title="What's the rating for                
                             generations-2?", description=None, required=True, type='rating',                      
                             values=[1, 2, 3, 4, 5]),                                                              
                             RemoteTextQuestion(id=UUID('685d4536-d7bf-4af8-bb1e-484a6eb71dd9'),                   
                             client=None, name='rating-rationale', title="What's the rationale behind              
                             each rating?", description=None, required=True, type='text',                          
                             use_markdown=False)]                                                                  
                                guidelines=None                                                                    
                                metadata_properties=[RemoteIntegerMetadataProperty(id=UUID('ee95c0e0-              
                             7b6b-433c-b360-58da9e5ac43f'), client=<httpx.Client object at                         
                             0x1a0b347

RemoteFeedbackDataset(
   id=667c5d11-5620-4d89-b3d4-03ba90278766
   name=notus_AI_preference
   workspace=Workspace(id=3ccf7c31-bab2-4091-8d01-c0ffa6e605db, name=admin, inserted_at=2023-12-18 08:21:20.790817, updated_at=2023-12-18 08:21:20.790817)
   url=https://ignacioct-argilla.hf.space/dataset/667c5d11-5620-4d89-b3d4-03ba90278766/annotation-mode
   fields=[RemoteTextField(id=UUID('a60fd7af-02b3-4e7b-8522-41c2c177b711'), client=None, name='input', title='input', required=True, type='text', use_markdown=False), RemoteTextField(id=UUID('85f037c3-75fe-4aa8-91a1-4a4863e4dcb7'), client=None, name='generations-1', title='generations-1', required=True, type='text', use_markdown=False), RemoteTextField(id=UUID('68262ba8-5141-4e99-bab3-a48d4234bab2'), client=None, name='generations-2', title='generations-2', required=True, type='text', use_markdown=False)]
   questions=[RemoteRatingQuestion(id=UUID('d630657a-575c-4716-a1bb-299935bcf076'), client=None, name='generations-1-rating', title="What